In [1]:
import numpy as np
import torch
import os
import gc
from sklearn import preprocessing

In [2]:
path = '/home/jua/corsika/run/energy_var/npz/square/all/'
filelist = os.listdir(path)
gbins = {}
pbins = {}
for file in filelist:
    n = int(file.split('_')[2][-1])
    if file.split("_")[0]=='gamma':
        gbins[n]=np.load(path+file)
    if file.split("_")[0]=='proton':
        pbins[n]=np.load(path+file)

In [3]:
def norm(Ebin, g, p, normtype):
    g_label = torch.zeros([len(g)])
    p_label = torch.ones([len(p)])
    if normtype == "minmax":
        min_max_scaler = preprocessing.MinMaxScaler()
        for j in range(len(g)):
            for i in range(4):
                g[j][i] = min_max_scaler.fit_transform(g[j][i])
        for j in range(len(p)):
            for i in range(4):
                p[j][i] = min_max_scaler.fit_transform(p[j][i])
    elif normtype == "std":
        for j in range(len(g)):
            for i in range(4):
                scaler = preprocessing.StandardScaler().fit(g[j][i])
                g[j][i] = scaler.transform(g[j][i])
        for j in range(len(p)):
            for i in range(4):
                scaler = preprocessing.StandardScaler().fit(p[j][i])
                p[j][i] = scaler.transform(p[j][i])
    images = np.vstack((g,p))
    labels = np.hstack((g_label, p_label))
    np.savez_compressed("/home/jua/SWGO/square/data/{}/bin/bin{}.npz".format(normtype, Ebin),image=images,label=labels)
    return g, p

In [4]:
def flip(Ebin, g, p, normtype):
    g_label = torch.zeros([len(g)])
    p_label = torch.ones([len(p)])
    
    g_ud = torch.zeros(len(g),4,51,51)
    p_ud = torch.zeros(len(p),4,51,51)
    
    g_rl = torch.zeros(len(g),4,51,51)
    p_rl = torch.zeros(len(p),4,51,51)
    
    g = torch.tensor(g).float()
    p = torch.tensor(p).float()
    
    for j in range(len(g)):
        for i in range(4):
            # Upside Down 
            g_ud[j][i] = torch.flip(g[j][i],[0])
            # Left and right reverse
            g_rl[j][i] = torch.flip(g[j][i],[1])
    
    for j in range(len(p)):
        for i in range(4):
            # Upside Down 
            p_ud[j][i] = torch.flip(p[j][i],[0])
            # Left and right reverse
            p_rl[j][i] = torch.flip(p[j][i],[1])
    images_ud = np.vstack((g_ud,p_ud))
    images_rl = np.vstack((g_rl,p_rl))
    labels = np.hstack((g_label, p_label))
    np.savez_compressed("/home/jua/SWGO/square/data/{}/augmentation/flip/updown_bin{}.npz".format(normtype, Ebin)
                        ,image=images_ud,label=labels)
    np.savez_compressed("/home/jua/SWGO/square/data/{}/augmentation/flip/rightleft_bin{}.npz".format(normtype, Ebin)
                        ,image=images_rl,label=labels)
    return len(g), len(p)

In [5]:
# anti clockwise
def rotate(Ebin, g, p, normtype):
    g_label = torch.zeros([len(g)])
    p_label = torch.ones([len(p)])
    
    g_90 = torch.zeros(len(g),4,51,51)
    p_90 = torch.zeros(len(p),4,51,51)
    
    g_180 = torch.zeros(len(g),4,51,51)
    p_180 = torch.zeros(len(p),4,51,51)
    
    g_270 = torch.zeros(len(g),4,51,51)
    p_270 = torch.zeros(len(p),4,51,51)
    
    g = torch.tensor(g).float()
    p = torch.tensor(p).float()
    
    for j in range(len(g)):
        for i in range(4):
            # 90 
            g_90[j][i] = torch.rot90(g[j][i],1)
            # 180
            g_180[j][i] = torch.rot90(g[j][i],2)
            # 270
            g_270[j][i] = torch.rot90(g[j][i],3)    
    for j in range(len(p)):
        for i in range(4):
            # 90 
            p_90[j][i] = torch.rot90(p[j][i],1)
            # 180
            p_180[j][i] = torch.rot90(p[j][i],2)
            # 270
            p_270[j][i] = torch.rot90(p[j][i],3)
    images_90 = np.vstack((g_90,p_90))
    images_180 = np.vstack((g_180,p_180))
    images_270 = np.vstack((g_270,p_270))
    labels = np.hstack((g_label, p_label))
    np.savez_compressed("/home/jua/SWGO/square/data/{}/augmentation/rotation/rot90_bin{}.npz".format(normtype, Ebin)
                        ,image=images_90,label=labels)
    np.savez_compressed("/home/jua/SWGO/square/data/{}/augmentation/rotation/rot180_bin{}.npz".format(normtype, Ebin)
                        ,image=images_180,label=labels)
    np.savez_compressed("/home/jua/SWGO/square/data/{}/augmentation/rotation/rot270_bin{}.npz".format(normtype, Ebin)
                        ,image=images_270,label=labels)
    return len(g), len(p)

In [6]:
for i in range(2,9):
    norm(i,gbins[i]['image'][:],pbins[i]['image'][:],'std')
    #gim, pim = norm(i,gbins[i]['image'][:],pbins[i]['image'][:],'std')
    #flip(i, gim, pim, 'std')
    #rotate(i, gim, pim, 'std')
    #del gim, pim
    #gc.collect()

In [ ]:
for i in range(1,9):
    norm(i,gbins[i]['image'][:],pbins[i]['image'][:],'minmax')
    '''
    gim, pim = norm(i,gbins[i]['image'][:],pbins[i]['image'][:],'minmax')
    flip(i, gim, pim, 'minmax')
    rotate(i, gim, pim, 'minmax')
    del gim, pim
    gc.collect()
    '''